In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from gensim import models, corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from gensim.models.coherencemodel import CoherenceModel
import time
import json
from nltk.corpus import stopwords
from math import log
from itertools import combinations
from tqdm import tqdm
import logging
from collections import Counter, defaultdict
import gensim
from gensim import corpora
from scipy.sparse import csr_matrix
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from transformers import BertTokenizer, BertModel
from bertopic import BERTopic
import seaborn as sns
from scipy import stats
import os
import re
import matplotlib
from tabulate import tabulate
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# NLTK 데이터 다운로드
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# stop_words 정의
stop_words = set(stopwords.words('english'))

# 전역 변수로 BERT 모델과 토크나이저 선언
global tokenizer, bert_model

# BERT 모델 로딩 함수
def load_bert_model():
    global tokenizer, bert_model
    if 'tokenizer' not in globals() or 'bert_model' not in globals():
        logging.info("Loading BERT model and tokenizer...")
        try:
            tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
            bert_model = BertModel.from_pretrained('bert-base-uncased')
            
            # Check GPU availability and set device
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            bert_model = bert_model.to(device)
            
            logging.info(f"BERT model and tokenizer loaded. Using device: {device}")
        except Exception as e:
            logging.error(f"Error loading BERT model: {e}")
            raise

def load_data(file_path, sample_size=20000, min_words=10):
    try:
        df = pd.read_csv(file_path, header=None, names=['text'])
        texts = df['text'].astype(str)
        
        # 최소 단어 수 조건을 만족하는 텍스트만 필터링
        valid_texts = texts[texts.apply(lambda x: len(x.split()) >= min_words)]
        total_samples = len(valid_texts)
        
        if total_samples > sample_size:
            sampled_texts = valid_texts.sample(n=sample_size, random_state=42)
        else:
            sampled_texts = valid_texts
            
        print(f"Loaded {len(sampled_texts)} texts from {file_path} (Total valid samples: {total_samples})")
        return sampled_texts.tolist()
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        return []
    except Exception as e:
        logging.error(f"Error loading file {file_path}: {e}")
        return []


def load_all_datasets():
    datasets = {
        'academy': {
            'covid': load_data('data/academy/covid.csv')
        },
        'media': {
            'clothing_review': load_data('data/media/clothing_review.csv')
        },
        'news': {
            'agnews': load_data('data/news/agnews.csv')
        }
    }
    return datasets

def print_dataset_statistics(datasets):
    print("\nSampled Dataset Statistics:")
    print("===========================")
    for domain, domain_data in datasets.items():
        print(f"\nDomain: {domain}")
        for dataset_name, data in domain_data.items():
            print(f"  Dataset: {dataset_name}")
            
            # 데이터가 이미 문자열의 리스트라고 가정
            print(f"    Sampled texts: {len(data)}")
            
            # 텍스트 길이 통계 (샘플링된 데이터)
            text_lengths = [len(text.split()) for text in data]
            print(f"    Average text length (words): {np.mean(text_lengths):.2f}")
            print(f"    Median text length (words): {np.median(text_lengths):.2f}")
            print(f"    Min text length (words): {np.min(text_lengths)}")
            print(f"    Max text length (words): {np.max(text_lengths)}")
            
            # 고유 단어 수 (샘플링된 데이터)
            unique_words = set(word for text in data for word in text.split())
            print(f"    Unique words: {len(unique_words)}")


class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim=50, latent_dim=None):
        if latent_dim is None:
            raise ValueError("latent_dim must be specified")
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, latent_dim)  
        self.fc22 = nn.Linear(hidden_dim, latent_dim)  
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, input_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

def vae_loss(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

def extract_vae_topics(vae_model, vectorizer, num_topics, top_n=10):
    with torch.no_grad():
        latent_vectors = torch.eye(num_topics).to(vae_model.fc3.weight.device)
        decoder_output = vae_model.decode(latent_vectors)
        decoder_output = decoder_output.cpu().numpy()

    feature_names = vectorizer.get_feature_names_out()
    topics = []
    for topic_distribution in decoder_output:
        top_indices = topic_distribution.argsort()[-top_n:][::-1]
        topic_words = [feature_names[i] for i in top_indices]
        topics.append(topic_words)
    return topics

def perform_bertopic_modeling(data, max_topics=20):
    try:
        # Data preprocessing: Remove empty or non-string documents
        data = [doc for doc in data if isinstance(doc, str) and len(doc.strip()) > 0]
        if len(data) == 0:
            logging.error("No valid documents to process.")
            return None, None, None
        
        # Adjust min_df to 1
        vectorizer_model = CountVectorizer(stop_words="english", min_df=1, max_df=0.95)
        
        # Initialize BERTopic
        bertopic_model = BERTopic(
            language="english",
            calculate_probabilities=True,
            nr_topics="auto",
            vectorizer_model=vectorizer_model,
            top_n_words=10,
            min_topic_size=30,
            n_gram_range=(1, 2),
            verbose=True
        )
        
        # Fit the model
        topics, _ = bertopic_model.fit_transform(data)
        
        # Reduce topics if necessary
        num_topics = len(set(topics)) - (1 if -1 in topics else 0)
        if num_topics > max_topics:
            logging.info(f"Reducing number of topics from {num_topics} to {max_topics}")
            bertopic_model = bertopic_model.reduce_topics(data, nr_topics=max_topics)
            topics = bertopic_model.transform(data)
            num_topics = len(set(topics)) - (1 if -1 in topics else 0)
        
        # Extract topic words
        topic_words = []
        for i in range(num_topics):
            topic = bertopic_model.get_topic(i)
            if topic:
                words = [word for word, _ in topic[:10]]
                topic_words.append(words)
        
        logging.info(f"BERTopic modeling completed. Number of topics: {num_topics}")
        return bertopic_model, topic_words, num_topics
    except Exception as e:
        logging.error(f"Error in BERTopic modeling: {e}")
        return None, [], 0  # 빈 리스트와 0을 반환



def perform_lowbertopic_modeling(data, low_num_topics):
    # 토픽 수를 num_topics로 설정 (기본값 5)
    low_num_topics = 5
    
    # 데이터 전처리
    if isinstance(data, dict):
        data = list(data.values())[0]
    elif isinstance(data, pd.DataFrame):
        data = data['text'].tolist() if 'text' in data.columns else data.values.flatten().tolist()
    elif isinstance(data, pd.Series):
        data = data.tolist()
    elif isinstance(data, np.ndarray):
        data = data.flatten().tolist()
    elif isinstance(data, list):
        pass
    else:
        raise ValueError(f"Unsupported data format for BERTopic modeling: {type(data)}")

    # 데이터가 문자열 리스트인지 확인
    if not all(isinstance(item, str) for item in data):
        raise ValueError("All items in the data must be strings")

    try:
        
        # BERTopic 모델 생성
        lowbertopic_model = BERTopic(language="english", calculate_probabilities=True, nr_topics=low_num_topics)
        
        # 모델 학습
        topics, _ = lowbertopic_model.fit_transform(data)
        
        # 토픽 단어 추출
        topic_words = []
        for i in range(low_num_topics):
            topic = lowbertopic_model.get_topic(i)
            if topic:
                words = [word for word, _ in topic[:10]]  # 상위 10개 단어 추출
                topic_words.append(words)
        
        return lowbertopic_model, topic_words, low_num_topics
    except Exception as e:
        logging.error(f"Error in LowBERTopic modeling: {e}")
        return None, None, None


def perform_vae_topic_modeling(data, num_topics, num_epochs=5, hidden_dim=50):
    try:
        # 데이터 전처리
        data = [str(doc) for doc in data if isinstance(doc, str) and len(doc) > 0]
        vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
        doc_term_matrix = vectorizer.fit_transform(data)

        # MinMaxScaler를 사용하여 0-1 사이로 정규화
        scaler = MinMaxScaler()
        normalized_matrix = scaler.fit_transform(doc_term_matrix.toarray())

        # VAE 모델 초기화 및 학습
        input_dim = doc_term_matrix.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        vae_model = VAE(input_dim=input_dim, hidden_dim=hidden_dim, latent_dim=num_topics).to(device)
        optimizer = torch.optim.Adam(vae_model.parameters(), lr=1e-3)

        batch_size = 64
        data_loader = DataLoader(normalized_matrix.astype(np.float32), batch_size=batch_size, shuffle=True)

        vae_model.train()
        for epoch in range(num_epochs):
            train_loss = 0
            for batch in data_loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                recon_batch, mu, logvar = vae_model(batch)
                loss = vae_loss(recon_batch, batch, mu, logvar)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            logging.info(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss / len(data_loader.dataset):.4f}")

        topics = extract_vae_topics(vae_model, vectorizer, num_topics)
        return vae_model, topics
    except Exception as e:
        logging.error(f"Error in VAE modeling: {e}")
        return None, None

def calculate_coherence(topics, tokenizer, bert_model, top_n=10, batch_size=16):
    coherence_scores = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    bert_model.to(device)
    bert_model.eval()

    for topic_words in topics:
        # 토픽 단어 수 제한
        topic_words = topic_words[:top_n]
        # 유효한 단어만 선택
        topic_words = [word for word in topic_words if word and isinstance(word, str)]
        num_words = len(topic_words)

        if num_words < 2:
            coherence_scores.append(0)
            continue

        # 단어 임베딩을 배치로 계산     
        embeddings = []
        for i in range(0, num_words, batch_size):
            batch_words = topic_words[i:i + batch_size]
            inputs = tokenizer(batch_words, return_tensors="pt", padding=True, truncation=True).to(device)
            with torch.no_grad():
                outputs = bert_model(**inputs)
            batch_embeddings = outputs.last_hidden_state[:, 0, :]  # [CLS] 토큰의 임베딩 사용
            embeddings.append(batch_embeddings)
            # 메모리 관리
            del outputs
            torch.cuda.empty_cache()

        embeddings = torch.cat(embeddings, dim=0)

        # 코사인 유사도 계산
        pairwise_similarities = []
        for i in range(num_words):
            for j in range(i + 1, num_words):
                cosine_sim = torch.nn.functional.cosine_similarity(embeddings[i], embeddings[j], dim=0)
                pairwise_similarities.append(cosine_sim.item())

        coherence = np.mean(pairwise_similarities)
        coherence_scores.append(coherence)

    final_coherence = np.mean(coherence_scores) if coherence_scores else 0
    return final_coherence


def process_metrics(domain, model_type, topics, data, metrics_list, tokenizer, bert_model):
    tokenized_data = [simple_preprocess(doc) for doc in data]
    dictionary = Dictionary(tokenized_data)
    corpus = [dictionary.doc2bow(text) for text in tokenized_data]

    coherence = calculate_coherence(topics, tokenizer, bert_model)
    npmi = calculate_npmi(topics, corpus, dictionary)
    umass = calculate_umass(topics, corpus, dictionary)

    metrics_list.append({
        'Domain': domain,
        'Model': model_type,
        'Coherence': coherence,
        'NPMI': npmi,
        'U_Mass': umass
    })

    logging.info(f"Coherence: {coherence:.4f}, NPMI: {npmi:.4f}, U_Mass: {umass:.4f}")
    
    return [metrics_list[-1]]  # Return the last added metric as a list

def calculate_npmi(topics, corpus, dictionary, top_n=10):
    # Create a set of all words used in topics
    topic_words_set = set()
    for topic in topics:
        topic_words_set.update(topic[:top_n])

    # Map words to IDs
    word2id = {word: dictionary.token2id[word] for word in topic_words_set if word in dictionary.token2id}
    id2word = {id: word for word, id in word2id.items()}

    # Calculate word and word pair document frequencies
    total_docs = len(corpus)
    word_doc_freq = defaultdict(int)
    pair_doc_freq = defaultdict(int)

    for doc in corpus:
        doc_word_ids = set([id for id, _ in doc])     
        topic_word_ids_in_doc = doc_word_ids.intersection(set(word2id.values()))

        for word_id in topic_word_ids_in_doc:
            word_doc_freq[word_id] += 1

        for word_id1, word_id2 in combinations(topic_word_ids_in_doc, 2):
            pair = tuple(sorted((word_id1, word_id2)))
            pair_doc_freq[pair] += 1

    # Calculate NPMI
    npmi_scores = []
    for topic in topics:
        topic_word_ids = [word2id[word] for word in topic[:top_n] if word in word2id]
        if len(topic_word_ids) < 2:
            continue
        pair_npmi_scores = []
        for word_id1, word_id2 in combinations(topic_word_ids, 2):
            pair = tuple(sorted((word_id1, word_id2)))
            co_doc_count = pair_doc_freq.get(pair, 0)
            if co_doc_count == 0:
                continue
            p_w1_w2 = co_doc_count / total_docs
            p_w1 = word_doc_freq[word_id1] / total_docs
            p_w2 = word_doc_freq[word_id2] / total_docs

            pmi = np.log(p_w1_w2 / (p_w1 * p_w2) + 1e-12)
            npmi = pmi / (-np.log(p_w1_w2 + 1e-12))
            pair_npmi_scores.append(npmi)
        if pair_npmi_scores:
            npmi_scores.append(np.mean(pair_npmi_scores))

    return np.mean(npmi_scores) if npmi_scores else float('nan')

def calculate_umass(topics, corpus, dictionary, top_n=10):
    # Create a set of all words used in topics
    topic_words_set = set()
    for topic in topics:
        topic_words_set.update(topic[:top_n])

    # Map words to IDs
    word2id = {word: dictionary.token2id[word] for word in topic_words_set if word in dictionary.token2id}

    # Calculate word and word pair frequencies
    word_counts = defaultdict(int)
    pair_counts = defaultdict(int)

    for doc in corpus:
        doc_word_ids = set([id for id, _ in doc])
        topic_word_ids_in_doc = doc_word_ids.intersection(set(word2id.values()))

        for word_id in topic_word_ids_in_doc:
            word_counts[word_id] += 1

        for word_id1, word_id2 in combinations(topic_word_ids_in_doc, 2):
            pair = tuple(sorted((word_id1, word_id2)))
            pair_counts[pair] += 1

    # Calculate U_Mass
    umass_scores = []
    for topic in topics:
        topic_word_ids = [word2id[word] for word in topic[:top_n] if word in word2id]
        if len(topic_word_ids) < 2:
            continue
        pair_umass_scores = []
        for i, word_id1 in enumerate(topic_word_ids):
            for word_id2 in topic_word_ids[:i]:
                pair = tuple(sorted((word_id1, word_id2)))
                co_occurrence = pair_counts.get(pair, 0) + 1  # Add 1 for smoothing
                word2_count = word_counts[word_id2] + 1  # Add 1 for smoothing
                umass = np.log(co_occurrence / word2_count)
                pair_umass_scores.append(umass)
        if pair_umass_scores:
            umass_scores.append(-np.mean(pair_umass_scores))

    return -np.mean(umass_scores) if umass_scores else float('nan')

# 일치도 분석 함수
def analyze_agreement(metrics_df):
    agreement_results = {}
    
    # Add debugging information
    print("DataFrame columns:", metrics_df.columns)
    print("DataFrame head:")
    print(metrics_df.head())
    
    for domain in metrics_df['Domain'].unique():
        domain_df = metrics_df[metrics_df['Domain'] == domain]
        for model in ['BERTopic', 'VAE', 'LowBERTopic']:
            model_df = domain_df[domain_df['Model'] == model]
            if len(model_df) == 0:
                continue
            
            # Check if the required columns exist
            required_columns = ['Coherence', 'NPMI', 'U_Mass']
            if not all(col in model_df.columns for col in required_columns):
                print(f"Warning: Missing required columns for {domain} - {model}")
                continue
            
            coherence = model_df['Coherence'].values[0]
            npmi = model_df['NPMI'].values[0]
            umass = model_df['U_Mass'].values[0]
            
            # Calculate agreement between metrics
            metrics = [coherence, npmi, -umass]  # Invert U_Mass as lower is better
            agreement = np.std(metrics) / np.mean(metrics)  # Coefficient of Variation
            
            if domain not in agreement_results:
                agreement_results[domain] = {}
            agreement_results[domain][model] = {
                'Coherence': coherence,
                'NPMI': npmi,
                'U_Mass': umass,
                'Agreement': agreement
            }
    
    return agreement_results

# 안정성 분석 함수
def analyze_stability(datasets, model_types, n_runs=10, sample_ratio=0.8):
    stability_results = []
    
    for domain, domain_datasets in datasets.items():
        # 각 도메인에서 첫 번째 데이터셋만 사용
        data = next(iter(domain_datasets.values()))
        
        logging.info(f"Analyzing stability for domain: {domain}")
        logging.info(f"Original data type: {type(data)}")
        
        # 데이터 형식 확인 및 변환
        if isinstance(data, pd.DataFrame):
            data = data['text'].tolist() if 'text' in data.columns else data.values.flatten().tolist()
        elif isinstance(data, pd.Series):
            data = data.tolist()
        elif isinstance(data, np.ndarray):
            data = data.flatten().tolist()
        elif isinstance(data, list):
            pass
        else:
            raise ValueError(f"Unsupported data format for domain {domain}: {type(data)}")
        
        logging.info(f"Processed data type: {type(data)}")
        logging.info(f"Sample of processed data: {data[:5]}")  # 처음 5개 항목 출력
        
        # BERTopic으로 초기 토픽 수 결정
        _, _, num_topics = perform_bertopic_modeling(data)
        
        for model_type in model_types:
            metric_values = {
                'Coherence': [],
                'NPMI': [],
                'U_Mass': []
            }
            
            for _ in range(n_runs):
                sampled_data = np.random.choice(data, size=int(len(data) * sample_ratio), replace=False)
                sampled_data = sampled_data.tolist()  # numpy array를 리스트로 변환
                
                logging.info(f"Sampled data type: {type(sampled_data)}")
                logging.info(f"Sample of sampled data: {sampled_data[:5]}")  # 처음 5개 항목 출력
                
                if model_type == 'BERTopic':
                    model, topics, _ = perform_bertopic_modeling(sampled_data)
                elif model_type == 'VAE':
                    model, topics = perform_vae_topic_modeling(sampled_data, num_topics)
                elif model_type == 'LowBERTopic':
                    model, topics, _ = perform_lowbertopic_modeling(sampled_data, num_topics)  # num_topics 인자 제거
                else:
                    raise ValueError(f"Unsupported model type: {model_type}")
                
                if model is None or topics is None:
                    logging.warning(f"Model or topics is None for {model_type} in domain {domain}")
                    continue
                
                tokenized_data = [simple_preprocess(doc) for doc in sampled_data]
                dictionary = Dictionary(tokenized_data)
                corpus = [dictionary.doc2bow(text) for text in tokenized_data]
                
                coherence = calculate_coherence(topics, tokenizer, bert_model)
                npmi = calculate_npmi(topics, corpus, dictionary)
                umass = calculate_umass(topics, corpus, dictionary)
                
                metric_values['Coherence'].append(coherence)
                metric_values['NPMI'].append(npmi)
                metric_values['U_Mass'].append(umass)
            
            for metric, values in metric_values.items():
                cv = np.std(values) / np.mean(values) if np.mean(values) != 0 else float('nan')
                stability_results.append({
                    'Domain': domain,
                    'Model': model_type,
                    'Metric': metric,
                    'CV': cv
                })
    
    return pd.DataFrame(stability_results)

# 개선된 토픽 품질 시각화 함수

def visualize_topic_quality(metrics_df):
    plt.figure(figsize=(12, 8))
    
    # Add debugging information
    print("DataFrame columns:", metrics_df.columns)
    print("DataFrame head:")
    print(metrics_df.head())
    
    # Check if required columns exist
    required_columns = ['Model', 'Coherence', 'NPMI']
    if not all(col in metrics_df.columns for col in required_columns):
        print(f"Error: Missing required columns. Available columns: {metrics_df.columns}")
        return
    
    # Use different markers for each model
    markers = {'BERTopic': 'o', 'VAE': 's', 'LowBERTopic': '^'}
    
    for model in ['BERTopic', 'VAE', 'LowBERTopic']:
        model_data = metrics_df[metrics_df['Model'] == model]
        coherence = model_data['Coherence']
        npmi = model_data['NPMI']
        plt.scatter(coherence, npmi, label=model, marker=markers[model])
    
    plt.xlabel('Coherence')
    plt.ylabel('NPMI')
    plt.title('Topic Quality: Coherence vs NPMI')
    plt.legend()
    
    # Add domain labels
    for _, row in metrics_df.iterrows():
        plt.annotate(row['Domain'], (row['Coherence'], row['NPMI']), xytext=(5, 5), 
                     textcoords='offset points', fontsize=8)
    
    plt.tight_layout()
    plt.savefig('topic_quality.png')
    plt.close()
    
    logging.info("Topic quality visualization completed: topic_quality.png")


def evaluate_coherence_stability(models, domains, datasets, n_runs=5):
    stability_results = []
    
    for model in models:
        for domain, data in zip(domains, datasets):
            # Handle dictionary data appropriately
            if isinstance(data, dict):
                data = list(data.values())[0]
            elif isinstance(data, pd.DataFrame):
                data = data['text'].tolist()
            elif not isinstance(data, list):
                raise ValueError(f"Unsupported data format for domain {domain}")

            coherence_scores = []
            npmi_scores = []
            umass_scores = []

            for _ in range(n_runs):
                # Sample 80% of the data
                sampled_data = np.random.choice(data, size=int(len(data) * 0.8), replace=False)

                if model == 'BERTopic':
                    _, topics, _ = perform_bertopic_modeling(sampled_data)
                elif model == 'VAE':
                    _, topics = perform_vae_topic_modeling(sampled_data, num_topics=10)  # Adjust num_topics as needed
                elif model == 'LowBERTopic':
                    _, topics, _ = perform_lowbertopic_modeling(sampled_data, low_num_topics=5)  
                else:                    
                    raise ValueError(f"Unsupported model type: {model}")

                # Prepare tokenized data
                tokenized_data = [simple_preprocess(doc) for doc in sampled_data]
                dictionary = Dictionary(tokenized_data)
                corpus = [dictionary.doc2bow(text) for text in tokenized_data]

                # Calculate coherence metrics
                coherence = calculate_coherence(topics, tokenizer, bert_model)
                npmi = calculate_npmi(topics, corpus, dictionary)
                umass = calculate_umass(topics, corpus, dictionary)

                coherence_scores.append(coherence)
                npmi_scores.append(npmi)
                umass_scores.append(umass)

            # Calculate stability (coefficient of variation)
            coherence_stability = np.std(coherence_scores) / np.mean(coherence_scores)
            npmi_stability = np.std(npmi_scores) / np.mean(npmi_scores)
            umass_stability = np.std(umass_scores) / np.mean(umass_scores)

            stability_results.append({
                'Model': model,
                'Domain': domain,
                'Coherence_Stability': coherence_stability,
                'NPMI_Stability': npmi_stability,
                'UMass_Stability': umass_stability,
                'Mean_Coherence': np.mean(coherence_scores),
                'Mean_NPMI': np.mean(npmi_scores),
                'Mean_UMass': np.mean(umass_scores)
            })

    return pd.DataFrame(stability_results)

def print_results(metrics_df, agreement_results, stability_df, stability_results):
    logging.info("\n=== Results Analysis ===")
    
    # Print average performance of metrics by model
    logging.info("\nAverage performance of metrics by model:")
    models = metrics_df['Model'].unique()
    for model in models:
        model_metrics = metrics_df[metrics_df['Model'] == model]
        mean_metrics = model_metrics[['Coherence', 'NPMI', 'U_Mass']].mean()
        logging.info(f"\nModel: {model}")
        logging.info(f"  - Average Coherence: {mean_metrics['Coherence']:.4f}")
        logging.info(f"  - Average NPMI: {mean_metrics['NPMI']:.4f}")
        logging.info(f"  - Average U_Mass: {mean_metrics['U_Mass']:.4f}")
    
    # Print average performance of metrics by domain
    logging.info("\nAverage performance of metrics by domain:")
    domains = metrics_df['Domain'].unique()
    for domain in domains:
        domain_metrics = metrics_df[metrics_df['Domain'] == domain]
        mean_metrics = domain_metrics[['Coherence', 'NPMI', 'U_Mass']].mean()
        logging.info(f"\nDomain: {domain}")
        logging.info(f"  - Average Coherence: {mean_metrics['Coherence']:.4f}")
        logging.info(f"  - Average NPMI: {mean_metrics['NPMI']:.4f}")
        logging.info(f"  - Average U_Mass: {mean_metrics['U_Mass']:.4f}")
    
    # Print agreement analysis results
    logging.info("\nAgreement analysis results between metrics:")
    for domain, model_results in agreement_results.items():
        logging.info(f"\nDomain: {domain}")
        for model, metrics in model_results.items():
            logging.info(f"  Model: {model}")
            for metric, value in metrics.items():
                logging.info(f"    {metric}: {value:.4f}")
    
    # Print coherence metric stability results
    logging.info("\nIndividual results of coherence metric stability:")
    logging.info(stability_results)
    logging.info("\nOverall results of coherence metric stability:")
    stability_summary = stability_df.groupby(['Model', 'Metric'])['CV'].mean().reset_index()
    for _, row in stability_summary.iterrows():
        logging.info(f"Model: {row['Model']}, Metric: {row['Metric']}, Average CV: {row['CV']:.4f}")
    
    logging.info("\nAnalysis complete. Please review and interpret the results.")

def process_datasets(datasets):
    all_metrics = []
    bertopic_results = {}
    vae_results = {}
    lowbertopic_results = {}
    
    for domain, domain_datasets in datasets.items():
        for dataset_name, data in domain_datasets.items():
            
            # 데이터가 리스트가 아닌 경우 리스트로 변환
            if not isinstance(data, list):
                data = [data]
            
            # BERTopic modeling
            bertopic_model, bertopic_topics, num_topics = perform_bertopic_modeling(data, max_topics=20)
            if bertopic_topics is None:
                bertopic_topics = []
                num_topics = 0
            
            bertopic_results[domain] = {
                'num_topics': num_topics,
                'topics': bertopic_topics
            }
            
            # Calculate BERTopic metrics
            if bertopic_topics:
                bertopic_metrics = process_metrics(domain, 'BERTopic', bertopic_topics, data, [], tokenizer, bert_model)
                all_metrics.extend(bertopic_metrics)
            
            # VAE modeling (using the number of topics from BERTopic)
            vae_model, vae_topics = perform_vae_topic_modeling(data, num_topics)
            if vae_topics is None:
                vae_topics = []
            
            vae_results[domain] = {
                'num_topics': num_topics,
                'topics': vae_topics
            }
            
            # Calculate VAE metrics
            if vae_topics:
                vae_metrics = process_metrics(domain, 'VAE', vae_topics, data, [], tokenizer, bert_model)
                all_metrics.extend(vae_metrics)
            
            # LowBERTopic modeling
            lowbertopic_model, lowbertopic_topics, low_num_topics = perform_lowbertopic_modeling(data, low_num_topics=num_topics)
            if lowbertopic_topics is None:
                lowbertopic_topics = []
                low_num_topics = 0
            
            lowbertopic_results[domain] = {
                'num_topics': low_num_topics,
                'topics': lowbertopic_topics
            }
            
            # Calculate LowBERTopic metrics
            if lowbertopic_topics:
                lowbertopic_metrics = process_metrics(domain, 'LowBERTopic', lowbertopic_topics, data, [], tokenizer, bert_model)
                all_metrics.extend(lowbertopic_metrics)
    
    return all_metrics, bertopic_results, vae_results, lowbertopic_results
        
        
def main():
    try:
        logging.basicConfig(level=logging.INFO)
        load_bert_model()
        logging.info("Starting dataset loading")
        datasets = load_all_datasets()
        print_dataset_statistics(datasets)
        
        logging.info("Starting topic modeling and metric calculation")
        all_metrics, bertopic_results, vae_results, lowbertopic_results = process_datasets(datasets)
        
        # Create DataFrame to store topic information
        topics_df = pd.DataFrame(columns=['Domain', 'Model', 'Topics'])
        
        # Process and save BERTopic results
        logging.info("Outputting and saving BERTopic results")
        for domain, result in bertopic_results.items():
            print(f"\nDomain: {domain}")
            print(f"Number of BERTopic topics: {result['num_topics']}")
            print("BERTopic topics:")
            for i, topic in enumerate(result['topics']):
                print(f"  Topic {i+1}: {', '.join(topic[:10])}")
            
            # Add to topics_df
            topics_df = pd.concat([topics_df, pd.DataFrame({
                'Domain': [domain],
                'Model': ['BERTopic'],
                'Topics': [result['topics']]
            })], ignore_index=True)
        
        # Process and save VAE results
        logging.info("\nOutputting and saving VAE results")
        for domain, result in vae_results.items():
            print(f"\nDomain: {domain}")
            print(f"Number of VAE topics: {result['num_topics']}")
            print("VAE topics:")
            for i, topic in enumerate(result['topics']):
                print(f"  Topic {i+1}: {', '.join(topic[:10])}")
            
            # Add to topics_df
            topics_df = pd.concat([topics_df, pd.DataFrame({
                'Domain': [domain],
                'Model': ['VAE'],
                'Topics': [result['topics']]
            })], ignore_index=True)
        
        # Process and save LowBERTopic results
        logging.info("\nOutputting and saving LowBERTopic results")
        for domain, result in lowbertopic_results.items():
            print(f"\nDomain: {domain}")
            print(f"Number of LowBERTopic topics: {result['num_topics']}")
            print("LowBERTopic topics:")
            for i, topic in enumerate(result['topics']):
                print(f"  Topic {i+1}: {', '.join(topic[:10])}")
            
            # Add to topics_df
            topics_df = pd.concat([topics_df, pd.DataFrame({
                'Domain': [domain],
                'Model': ['LowBERTopic'],
                'Topics': [result['topics']]
            })], ignore_index=True)
        
        # Save topics_df to CSV file
        topics_df.to_csv('topics_df.csv', index=False)
        logging.info("Topic information has been saved to topics_df.csv file.")
        
        logging.info("Starting metric analysis")
        metrics_df = pd.DataFrame(all_metrics)
        
        # U_Mass 값의 부호를 바꿉니다
        metrics_df['U_Mass'] = -metrics_df['U_Mass']
        logging.info("U_Mass values have been inverted for consistent interpretation with other metrics")
        
        metrics_df.to_csv('topic_modeling_metrics.csv', index=False)
        
        logging.info("Starting agreement analysis")
        agreement_results = analyze_agreement(metrics_df)
        
        logging.info("Starting stability analysis")
        stability_df = analyze_stability(datasets, ['BERTopic', 'VAE', 'LowBERTopic'])
        
        logging.info("Starting topic quality visualization")
        visualize_topic_quality(metrics_df)
        
        logging.info("Starting coherence stability evaluation")
        stability_results = evaluate_coherence_stability(['BERTopic', 'VAE', 'LowBERTopic'], list(datasets.keys()), list(datasets.values()))
        
        logging.info("Starting result output")
        print_results(metrics_df, agreement_results, stability_df, stability_results)
        
        logging.info("All analyses completed")
    except Exception as e:
        logging.error(f"Unexpected error occurred in main function execution: {e}")
        raise

# 메인 실행 부분
if __name__ == "__main__":
    main()

In [ ]:
 import os
import json
import logging
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from collections import Counter
from tqdm import tqdm
import anthropic
from tenacity import retry, stop_after_attempt, wait_random_exponential
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from nltk.corpus import stopwords
from sklearn.metrics import adjusted_rand_score

# BERT 모델과 토크나이저를 위한 import
from transformers import BertTokenizer, BertModel

# 필요한 경우 NLTK 데이터 다운로드
import nltk
nltk.download('stopwords', quiet=True)

# 로깅 설정 (첫 번째 셀에서 이미 설정되었을 수 있지만, 안전을 위해 다시 설정)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# stop_words 정의 (첫 번째 셀에서 이미 정의되었을 수 있지만, 안전을 위해 다시 정의)
stop_words = set(stopwords.words('english'))

# 전역 변수로 BERT 모델과 토크나이저 선언 (첫 번째 셀에서 이미 선언되었을 수 있음)
global tokenizer, bert_model

# matplotlib 백엔드 설정 (이미 설정되어 있을 수 있지만, 안전을 위해 다시 설정)
plt.switch_backend('agg')

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(3))
def call_anthropic_api(prompt: str, max_tokens_to_sample: int = 3000) -> str:
    anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
    if not anthropic_api_key:
        raise ValueError("Anthropic API key not found in environment variable ANTHROPIC_API_KEY")
    
    client = anthropic.Anthropic(api_key=anthropic_api_key)
    
    try:
        logging.info("Calling Anthropic API...")
        response = client.completions.create(
            model="claude-2",
            prompt=f"{anthropic.HUMAN_PROMPT} {prompt} {anthropic.AI_PROMPT}",
            max_tokens_to_sample=max_tokens_to_sample,
            temperature=0,
        )
        logging.info("Anthropic API call successful")
        return response.completion
    except Exception as e:
        logging.error(f"Error in Anthropic API call: {e}")
        logging.error(f"Client object: {client}")
        logging.error(f"Prompt: {prompt[:100]}...")  # Log only the first 100 characters
        raise

def llm_evaluation(topics):
    scores = []
    feedbacks = []

    prompt = f"""
    Evaluate the following topics based on their coherence. Coherence is an important metric for assessing the quality of topic modeling:

    1. Coherence measures how semantically related the words within each topic are.
    2. High coherence scores indicate that the words in a topic are closely related and form a meaningful theme.
    3. High coherence scores suggest that the words within a topic have strong semantic connections, making it easy for humans to understand and interpret the topic.
    4. Low coherence scores indicate that the words within a topic have little semantic relevance, potentially making the topic ambiguous and difficult to interpret.

    Please evaluate the following topics. For each topic, provide a coherence score on a scale of 1-10 and explain your reasoning:

    {topics}

    When evaluating, consider:
    1. How semantically related are the words within each topic?
    2. How clear and interpretable is the topic?
    3. Do the words in the topic represent a consistent theme or concept?

    Please respond for each topic in the following format exactly:
    Topic X: [score]
    Reason: [explanation]

    Note: Ensure the score is enclosed within square brackets, e.g., [7].
    """

    try:
        evaluation = call_anthropic_api(prompt)

        # Updated parsing logic to extract structured responses
        topic_evaluations = re.findall(r"Topic \d+:.*?(?=Topic \d+:|$)", evaluation, re.DOTALL)
        for eval in topic_evaluations:
            score_match = re.search(r'Topic (\d+):\s*\[?(\d+)\]?', eval)
            reason_match = re.search(r'Reason:\s*(.*)', eval, re.DOTALL)
            if score_match and reason_match:
                topic_score = int(score_match.group(2))
                if 1 <= topic_score <= 10:
                    scores.append(topic_score)
                    feedbacks.append(reason_match.group(1).strip())
                else:
                    print(f"Invalid score (not between 1 and 10): {eval}")
            else:
                print(f"Could not extract score or reason: {eval}")

    except Exception as e:
        print(f"Unexpected error: {e}")
        raise

    return scores, feedbacks

def run_llm_evaluation(sample_size=100, chunk_size=10):
    topics_df = pd.read_csv('topics_df.csv')
    llm_results = []
    actual_sample_size = min(sample_size, len(topics_df))
    
    for index, row in tqdm(topics_df.sample(n=actual_sample_size, random_state=42).iterrows(), total=actual_sample_size):
        domain = row['Domain']
        model_type = row['Model']
        topics = eval(row['Topics'])  # Convert string to list
        
        logging.info(f"LLM evaluation in progress - Domain: {domain}, Model: {model_type}")

        try:
            scores, feedbacks = llm_evaluation(topics)

            result = {
                'Domain': domain,
                'Model': model_type,
                'LLM_Scores': scores,
                'LLM_Feedbacks': feedbacks
            }
            llm_results.append(result)

            if len(llm_results) % chunk_size == 0:
                save_results_chunk(llm_results[-chunk_size:])
                
        except Exception as e:
            logging.error(f"Error processing {domain} - {model_type}: {str(e)}")
            continue

    if len(llm_results) % chunk_size != 0:
        save_results_chunk(llm_results[-(len(llm_results) % chunk_size):])

    llm_df = pd.DataFrame(llm_results)
    return llm_df
         
          
def analyze_llm_results(llm_df):
    llm_df['LLM_Avg_Score'] = llm_df['LLM_Scores'].apply(lambda scores: np.mean([s for s in scores if s is not None]))
    llm_df['LLM_Std_Score'] = llm_df['LLM_Scores'].apply(lambda scores: np.std([s for s in scores if s is not None]))
    llm_df['LLM_Median_Score'] = llm_df['LLM_Scores'].apply(lambda scores: np.median([s for s in scores if s is not None]))

    print("\nLLM Evaluation Results:")
    print(llm_df[['Domain', 'Model', 'LLM_Avg_Score', 'LLM_Std_Score', 'LLM_Median_Score']])

def llm_auto_metric_correlation(metrics_df, llm_df):
    merged_df = pd.merge(metrics_df, llm_df, on=['Domain', 'Model'])

    metric_names = ['Coherence', 'NPMI', 'U_Mass']
    for metric in metric_names:
        valid_idx = merged_df['LLM_Avg_Score'].notnull()
        
        # Pearson 및 Spearman 상관계수 계산
        pearson_corr, p_value_pearson = stats.pearsonr(merged_df.loc[valid_idx, metric], merged_df.loc[valid_idx, 'LLM_Avg_Score'])
        spearman_corr, p_value_spearman = stats.spearmanr(merged_df.loc[valid_idx, metric], merged_df.loc[valid_idx, 'LLM_Avg_Score'])
        
        # ARI 계산을 위한 데이터 준비
        auto_metric = merged_df.loc[valid_idx, metric].values
        llm_scores = merged_df.loc[valid_idx, 'LLM_Avg_Score'].values
        
        # 점수를 범주화 (예: 3개의 범주로 나누기)
        auto_categories = np.digitize(auto_metric, bins=np.linspace(min(auto_metric), max(auto_metric), 4))
        llm_categories = np.digitize(llm_scores, bins=np.linspace(min(llm_scores), max(llm_scores), 4))
        
        # ARI 계산
        ari = adjusted_rand_score(auto_categories, llm_categories)
        
        print(f"\nComparison between LLM evaluation scores and {metric}:")
        print(f"Pearson: correlation coefficient = {pearson_corr:.4f}, p-value = {p_value_pearson:.4f}")
        print(f"Spearman: correlation coefficient = {spearman_corr:.4f}, p-value = {p_value_spearman:.4f}")
        print(f"Adjusted Rand Index (ARI) = {ari:.4f}")

def verify_llm_consistency(topics, n_repeats=5):
    all_scores = []
    for _ in range(n_repeats):
        scores, _ = llm_evaluation(topics)
        all_scores.append(scores)
    all_scores = np.array(all_scores)
    std_scores = np.std(all_scores, axis=0)
    avg_std = np.mean(std_scores)
    cv_scores = std_scores / np.mean(all_scores, axis=0)
    avg_cv = np.mean(cv_scores)
    print(f"\nAverage standard deviation of LLM evaluation: {avg_std:.4f}")
    print(f"Average coefficient of variation (CV) of LLM evaluation: {avg_cv:.4f}")

def analyze_llm_feedback(llm_df):
    all_words = []
    for feedbacks in llm_df['LLM_Feedbacks']:
        for feedback in feedbacks:
            words = feedback.lower().split()
            all_words.extend([word for word in words if word not in stop_words])

    word_freq = Counter(all_words)
    print("\nMost frequent keywords in feedback:")
    for word, count in word_freq.most_common(10):
        print(f"{word}: {count}")

    coherence_keywords = ['coherent', 'consistent', 'related', 'connected', 'meaningful']
    print("\nFrequency of coherence-related keywords:")
    for keyword in coherence_keywords:
        print(f"{keyword}: {word_freq[keyword]}")

    positive_keywords = ['good', 'great', 'excellent', 'well', 'clear']
    negative_keywords = ['poor', 'bad', 'unclear', 'confusing', 'unrelated']
    
    positive_count = sum(word_freq[word] for word in positive_keywords)
    negative_count = sum(word_freq[word] for word in negative_keywords)
    
    print(f"\nNumber of positive feedback keywords: {positive_count}")
    print(f"Number of negative feedback keywords: {negative_count}")

    relationship_keywords = ['related', 'similar', 'overlapping', 'connected', 'distinct']          
              
    print("\nFrequency of topic relationship keywords:")
    for keyword in relationship_keywords:
        print(f"{keyword}: {word_freq[keyword]}")

    quality_keywords = ['coherent', 'meaningful', 'interpretable', 'clear', 'specific']
    print("\nFrequency of topic quality keywords:")
    for keyword in quality_keywords:
        print(f"{keyword}: {word_freq[keyword]}")

    scores = [score for scores in llm_df['LLM_Scores'] for score in scores if score is not None]
    print("\nDistribution of coherence scores:")
    print(f"Mean: {np.mean(scores):.2f}")
    print(f"Median: {np.median(scores):.2f}")
    print(f"Standard deviation: {np.std(scores):.2f}")
    print(f"Minimum: {np.min(scores):.2f}")
    print(f"Maximum: {np.max(scores):.2f}")

    print("\nAverage coherence score by model:")
    for model in llm_df['Model'].unique():
        model_scores = [score for scores, m in zip(llm_df['LLM_Scores'], llm_df['Model']) 
                        for score in scores if score is not None and m == model]
        print(f"{model}: {np.mean(model_scores):.2f}")

def visualize_llm_results(llm_df):
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='Model', y='LLM_Avg_Score', data=llm_df)
    plt.title('Distribution of LLM Evaluation Scores by Model')
    plt.savefig('llm_model_score_distribution.png')
    plt.close()

    plt.figure(figsize=(12, 6))
    sns.scatterplot(x='Model', y='LLM_Avg_Score', data=llm_df)
    plt.title('LLM Evaluation Scores by Model')
    plt.legend()
    plt.savefig('llm_model_score.png')
    plt.close()

def save_results_chunk(results_chunk):
    with open('llm_evaluation_results.json', 'a') as f:
        for result in results_chunk:
            json.dump(result, f)
            f.write('\n')

# Execute LLM evaluation
logging.info("Starting LLM evaluation")
metrics_df = pd.read_csv('topic_modeling_metrics.csv')

llm_df = run_llm_evaluation()
analyze_llm_results(llm_df)
visualize_llm_results(llm_df)

logging.info("Starting correlation analysis between LLM evaluation and automatic metrics")
llm_auto_metric_correlation(metrics_df, llm_df)

logging.info("Starting LLM feedback analysis")
analyze_llm_feedback(llm_df)

logging.info("LLM evaluation completed")